In [1]:
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType
import pandas as pd
from pydataset import data
from env import host, username, password


Acquire, first 3:

Read the case, department, and source data into their own spark dataframes.

Let's see how writing to the local disk works in spark:

1. Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json



2. Inspect your folder structure. What do you notice?

Creating the csv didn't produce any folders, only a single file. While the JSON produced a folder. The json files are organized with the names of each column in each row.

3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.



In [2]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/05 15:03:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/05 15:03:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df_dept = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)
df_dept

DataFrame[dept_division: string, dept_name: string, standardized_dept_name: string, dept_subject_to_SLA: string]

In [8]:
df_dept.head()

Row(dept_division='311 Call Center', dept_name='Customer Service', standardized_dept_name='Customer Service', dept_subject_to_SLA='YES')

In [4]:
df_dept.write.json("dept_json", mode="overwrite")

In [5]:
df_source = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)
df_source

DataFrame[source_id: string, source_username: string]

In [6]:
df_source.write.json("source_json", mode="overwrite")

In [14]:
df_case = spark.read.csv("case.csv", sep=",", header=True, inferSchema=True)
df_case

DataFrame[case_id: int, case_opened_date: string, case_closed_date: string, SLA_due_date: string, case_late: string, num_days_late: double, case_closed: string, dept_division: string, service_request_type: string, SLA_days: double, case_status: string, source_id: string, request_address: string, council_district: int]

In [16]:
df_case.write.json("case_json", mode="overwrite")

In [7]:
df_dept.schema

StructType([StructField('dept_division', StringType(), True), StructField('dept_name', StringType(), True), StructField('standardized_dept_name', StringType(), True), StructField('dept_subject_to_SLA', StringType(), True)])

In [9]:
df_source.schema

StructType([StructField('source_id', StringType(), True), StructField('source_username', StringType(), True)])

In [17]:
df_case.schema
#case_opened_date, case_closed_date, SLA_due_date, to datetype
#case_closed to boolean
#num_days_late to float
#sla_days to float

StructType([StructField('case_id', IntegerType(), True), StructField('case_opened_date', StringType(), True), StructField('case_closed_date', StringType(), True), StructField('SLA_due_date', StringType(), True), StructField('case_late', StringType(), True), StructField('num_days_late', DoubleType(), True), StructField('case_closed', StringType(), True), StructField('dept_division', StringType(), True), StructField('service_request_type', StringType(), True), StructField('SLA_days', DoubleType(), True), StructField('case_status', StringType(), True), StructField('source_id', StringType(), True), StructField('request_address', StringType(), True), StructField('council_district', IntegerType(), True)])

In [18]:
df_case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [10]:
#change source_id to integer in source

df_source = df_source.withColumn("source_id", col("source_id").cast("int"))

In [11]:
#change dept_subject_to_SLA to boolean

df_dept = df_dept.withColumn("case_closed", expr('dept_subject_to_SLA == "YES"')).withColumn("case_open", expr('dept_subject_to_SLA == "YES"'))

In [13]:
df_dept.show()

+--------------------+--------------------+----------------------+-------------------+-----------+---------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|case_closed|case_open|
+--------------------+--------------------+----------------------+-------------------+-----------+---------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|       true|     true|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|       true|     true|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|       true|     true|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|       true|     true|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|       true|     true|
|Code Enforcement ...|Code Enforcement ...|  DSD/Code Enforcement|                YES|       true|     true|
|Code Enforcement .

In [ ]:
case = spark.read.csv("case.csv", sep=",", header=True, inferSchema=True)
source = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)
dept = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)


In [ ]:
#2
